In [1]:
import numpy as np
import pandas as pd

%matplotlib osx
import matplotlib.pyplot as plt
import matplotlib as mp

# from m_gncspline import global_natural_spline
# from align_single_frame import align

from scipy.spatial.transform import Rotation as R
from gcvspline import GCVSmoothedNSpline, SmoothedNSpline

import glob
from scipy.signal import find_peaks

ImportError: dlopen(/usr/local/lib/python3.7/site-packages/gcvspline/_gcvspl.cpython-37m-darwin.so, 2): Library not loaded: /usr/local/opt/gcc/lib/gcc/8/libgfortran.5.dylib
  Referenced from: /usr/local/lib/python3.7/site-packages/gcvspline/_gcvspl.cpython-37m-darwin.so
  Reason: image not found

# Functions to Align Coords

In [2]:
def get_rotation_matrix(i_v, unit=None):
    # From http://www.j3d.org/matrix_faq/matrfaq_latest.html#Q38
    if unit is None:
        unit = [1.0, 0.0, 0.0]
    # Normalize vector length
    i_v /= np.linalg.norm(i_v)

    # Get axis
    uvw = np.cross(unit,i_v)

    # compute trig values - no need to go through arccos and back
    rcos = np.dot(i_v, unit)
    rsin = np.linalg.norm(uvw)

    #normalize and unpack axis
    if not np.isclose(rsin, 0):
        uvw /= rsin
    u, v, w = uvw

    # Compute rotation matrix - re-expressed to show structure
    return (
        rcos * np.eye(3) +
        rsin * np.array([
            [ 0, -w,  v],
            [ w,  0, -u],
            [-v,  u,  0]
        ]) +
        (1.0 - rcos) * uvw[:,None] * uvw[None,:]
    )

def fix_coords(stuff,origin,target,gravity):
    """stuff: a numpy array with data to be re-aligned to a new coordinate system where Z aligns with gravity
    origin: a 3d vector specifying the oriigin of the coordinate system
    target: a 3d vector specifying the x axis of the coordinate system (points from origin to target)
    gravity: a 3D unit vector that specifies the direction of gravity (down) in the original coordinate system"""
    
    
    rmat = get_rotation_matrix(gnorm, unit=[0.0,0.0,-1.0])

    oe = np.dot(origin, rmat)
    te = np.dot(target, rmat)

    stuff1 = np.copy(stuff)
    
    for i in np.arange(len(stuff)):
        stuff1[i,:] = np.dot(stuff[i,:].T, rmat)
        
    
    X = np.array([te[0,0] - oe[0,0], te[0,1]-oe[0,1],0])
    rmat1 = get_rotation_matrix(X ,unit=[1,0,0])
    
    stuff2 = np.copy(stuff)
    for i in np.arange(len(stuff)):
        stuff2[i,:] = np.dot(stuff1[i,:].T, rmat1)
    
    return stuff2

# Import head trajectories + align coordinates

In [3]:
ref = pd.read_csv('/Users/grahmich/Desktop/Kinematics Paper/Dendrelaphis NC Data.csv')
ref = ref.dropna(how='all')

tn_list = list(ref['tn'])
transitions = list(ref['transition'])
accelerations = list(ref['acceleration'])
starts = list(ref['start'])

transitions = [f-g for f,g in zip(transitions, starts)]
accelerations = [i-j for i,j in zip(accelerations, starts)]


ref = ref.dropna(axis=0,how='all')
ref.head()

,tn,ID,svl,type,gs_att,start,transition,acceleration,stop,reference,orig_x,targ_x,orig_y,targ_y,orig_z,targ_z
0,9,2018DP01,113.900000,inline,50.0,1,1401.0,1401,1487,1498,NaN,NaN,NaN,NaN,NaN,NaN
1,10,2018DP01,113.900000,inline,46.0,1,865.0,865,918,1,NaN,NaN,NaN,NaN,NaN,NaN
2,11,2018DP01,113.900000,inline,45.2,1,365.0,365,443,1,NaN,NaN,NaN,NaN,NaN,NaN
3,14,2018DP01,113.900000,inline,55.0,1,1494.0,1494,1585,847,NaN,NaN,NaN,NaN,NaN,NaN
4,42,2018DP02,127.443333,inline,53.0,8063,8712.0,8712,8757,4468,-0.342431,0.202844,-0.39909,-0.39909,0.091379,0.114363


In [7]:
#load the files
path = '/Users/grahmich/Desktop/Kinematics Paper/Head_Trajectories/*'
Files = glob.glob(path)
Files = [i for i in Files if i[-3:]=="csv"]
Files.sort()
len(Files)

118

In [8]:
#check: are the csv files in the folder corresponding to the trials in the reference DF?
#are both trial lists in the same order?
trials1 = []
for file in Files:
    tn = int(file[-23:-20])
    trials1.append(tn)
    
whoops = [i for i in trials1 if not i in tn_list]
whoops1 = [i for i in tn_list if not i in trials1]
whoops2 = [i-j for i,j in zip(tn_list,trials1)]
print(whoops)
print(whoops1)
print(max(whoops2))
print(min(whoops2))

[]
[]
0
0


In [10]:
rough = []
origins = []
targets = []


for ind in np.arange(len(trials1)):
    
    ID = Files[ind][-19:-11] #read in as a string.
    trial = tn_list[ind]
    data = pd.read_csv(Files[ind])
    
    svl = ref.loc[ind]['svl']/100 #cm to m
    start = ref.loc[ind]['start']
    end = ref.loc[ind]['stop']
    frame = ref.loc[ind]['reference']
    
   
    #pull out relevant points: gravity, origin and target ends. 
    
    ghigh = np.array(data.loc[[frame-1],['pt4_X','pt4_Y','pt4_Z']])
    glow = np.array(data.loc[[frame-1],['pt5_X','pt5_Y','pt5_Z']])
    grav = (ghigh-glow)[0]
    gnorm = grav/np.linalg.norm(grav)
    
    OE = np.array(data.loc[[frame-1],['pt2_X','pt2_Y','pt2_Z']])
    TE = np.array(data.loc[[frame-1],['pt3_X','pt3_Y','pt3_Z']])
    
    
    #nan check #1
    if np.isnan(np.sum(ghigh)):
        print(str(trial)+": ghigh has nans")
        continue
        
    #nan check # 2
    if np.isnan(np.sum(glow)):
        print(str(trial)+": glow has nans")
        continue
    
    #nan check 3
    if np.isnan(np.sum(OE)):
        print(str(trial)+": OE has nans")
        continue
        
    #nan check 4
    if np.isnan(np.sum(TE)):
        print(str(trial)+": glow has nans")
        continue 
        
    OE = OE.reshape(1,3)
    TE = TE.reshape(1,3)
    g1 = glow.reshape(1,3)
    g2 = ghigh.reshape(1,3)

    
    #crop head data to area of interest, between gap entry and landing.
    head_traj = np.array(data[['pt1_X','pt1_Y','pt1_Z']])
    head_traj = head_traj[start:end,:]
    
    #nan check 5
    if np.isnan(np.sum(head_traj)):
        print(str(trial)+": head_traj has nans")
        continue
    
    #align coordinates
    oe1 = fix_coords(OE,OE,TE,gnorm)
    te1 = fix_coords(TE,OE,TE,gnorm)
    g1a = fix_coords(g1,OE,TE,gnorm)
    g2a = fix_coords(g2,OE,TE,gnorm)

    
    head1 = fix_coords(head_traj,OE,TE,gnorm)
    rough.append(head1) #save a list of all the aligned head trajectories
    origins.append(oe1) #save a list of origin positions
    targets.append(te1) #save a list of target positions
    

In [11]:
for i in np.arange(4):
    plt.figure()
    plt.plot(rough[i][:,0],rough[i][:,2])
    plt.plot(origins[i][0,0],origins[i][0,2],'o')
    plt.plot(targets[i][0,0],targets[i][0,2],'o')
    plt.gca().set_aspect('equal')

# Smoothing

## Determine appropriate smoothing parameter

GCV spline function auto-generates a smoothing parameter (p) that meets the generalized cross-validatory spline criterion (see: Woltring, 1986). Using the manual spline smoothing, we can increase or decrease that smoothing by altering p. In this section, I use several different multipliers to see which gives appropriate results. Appropriate results are those in which no real movements are lost, but errors from digitizing are smoothed out. 

To assess this, I look at all the trials smoothed with the gcv default p and try to find the peak and trough associated with the final dynamic movements. Snakes do not generally have a wiggle at the peak of their lunge - its an upside down parabola. I then recorded which trials deviated from this according to the peak finder (i.e. no peak found at the top or multiple wiggles, etc). I checked them against each video to see if the wiggle was legitimate or not. 

I then experimented with different p multipliers until I found one which kept real wiggles, but smoothed out false wiggles. 

In [7]:
# smooth all 119 files
# create smoothed trajectories using manual GCV p value

smooth1 = []

for i in np.arange(len(rough)):
    head1 = rough[i]
    smoothp = np.empty(np.shape(head1))
    fr = 120
    t = [each/fr for each in np.arange(np.shape(head1)[0])]
    p = 1

    for d in [0,1,2]:
        x = head1[:,d]
        GCV_auto = GCVSmoothedNSpline(t, x) #p chosen by program; check it is a good value.
        pf = GCV_auto.p * p
        GCV_manual = SmoothedNSpline(t, x, p=pf) #set to a multiple of the auto-selected value
        x_smoothed = GCV_manual(t)
        smoothp[:,d] = x_smoothed
        
    smooth1.append(smoothp)

In [54]:
# graph X-Z position for  all 119 trials showing peaks and valleys of the final lunge
for t in np.arange(len(tn_list)):
    plt.figure()
    
    pos = smooth1[t]
    tops = find_peaks(pos[:,2])[0]
    bots = find_peaks(-pos[:tops[-1],2])[0]

    tf = transitions[t]
    af = accelerations[t]

    plt.suptitle(tn_list[t])

    plt.scatter(pos[:,0],pos[:,2],s=4)
    plt.scatter(pos[tops[-1],0],pos[tops[-1],2],label='peak')

    if len(bots)>0:
        plt.scatter(pos[bots[-1],0],pos[bots[-1],2],label='trough')

    plt.scatter(pos[tf,0],pos[tf,2],label='transition frame')
#     plt.scatter(pos[af,0],pos[af,2],label='acceleration frame')

    plt.gca().set_ylim(min(pos[:,2])-0.10,max(pos[:,2])+0.10) #give a little breathing room around the data
    plt.gca().set_aspect('equal')

    plt.legend(loc='lower center')
    plt.savefig("/Users/grahmich/Desktop/Jupyter Notebooks/Graphs/X-Z Trajectories - Smoothed/"+str(tn_list[t])+"_headtraj.pdf")
    plt.close()

NameError: name 'smooth1' is not defined

In [18]:
# based on the above graph, the following trials were assessed using the video: 
# 71, 86, 92, 102, 141, 164, 166, 171, 189, 193, 237

# p_opts = [1,100,200,300,400,500,600,700,800]
p_opts = [1,25,50,75,100,125,150,175,200]
# p_opts = [41,44,47,50,53,62,63,64,65]

high = []
low = []
smooths = []

ind = tn_list.index(237)
head1 = rough[ind]

for p in p_opts:    
    #create smoothed trajectories
    smoothp = np.empty(np.shape(head1))
    fr = 120
    t = [each/fr for each in np.arange(np.shape(head1)[0])]

    for d in [0,1,2]:
        x = head1[:,d]
        GCV_auto = GCVSmoothedNSpline(t, x) #p chosen by program
        pf = GCV_auto.p * p
        GCV_manual = SmoothedNSpline(t, x, p=pf) #set to a multiple of the auto-selected value
        x_smoothed = GCV_manual(t)
        smoothp[:,d] = x_smoothed
        
    tops = find_peaks(smoothp[:,2])[0]
    bots = find_peaks(-smoothp[:tops[-1],2])[0]
    
    high.append(tops[-1])
    low.append(bots[-1])
    smooths.append(smoothp)

In [21]:
fig, ax = plt.subplots(3,3,sharex=True, sharey=True)

for i in [0,1,2]:
    for j in [0,1,2]:
        t = j+3*i
        
        tf = transitions[ind]
        ax[i,j].scatter(smooths[t][:,0],smooths[t][:,2], s = 4)
        # ax[i,j].scatter(smooths[t][high[t],0],smooths[t][high[t],2],label='high point')
        # ax[i,j].scatter(smooths[t][low[t],0],smooths[t][low[t],2],label='low point')
        # ax[i,j].scatter(smooths[t][tf,0],smooths[t][tf,2],label='transition frame')
        ax[i,j].set_title(p_opts[t])
        ax[i,j].set_aspect('equal')
        
        print(np.max(smooths[t][:,0]))
        
        fig.suptitle(tn_list[ind])

0.15762491568794815
0.15738036980994044
0.1573186054182394
0.15728051915430566
0.1572498858574879
0.15722240594790354
0.15719652587551167
0.15717157970257123
0.15714724772291794


Based on video and graphs of trial 71, smoothing parameter can be as much as 800 without losing wriggle after transition frame (real feature)

Based on video and graphs of trial 89, smoothing parameter can be as much as 800 without losing real features (peaks on way up)

Based on video and graphs of trial 92, smoothing parameter should be <100~200, because loses a real feature (nose tipping up)

Based on video and graphs of trial 102, smoothing parameter can be as much as 200 without losing real features.

Based on video and graphs of trial 141, smoothing parameter should be >25~50 to remove false wiggle in data (wiggle in final descent). 

Based on video and graphs of trial 164, smoothing parameter can be as much as 200 without losing real features.

Based on video and graphs of trial 166, smoothing parameter needs to be > 1~25 to remove false wiggle in data. 

Based on video and graphs of trial 171, smoothing parameter can be as much as 200 without losing real features.

Based on video and graphs of trial 189, smoothing parameter needs to be > 1~25 to remove false wiggle in data. 

Based on video and graphs of trial 193, smoothing parameter needs to be > 1~25 to remove false wiggle in data. 

Based on video and graphs of trial 237, smoothing parameter needs to be 41<p<65 to remove false wiggle in data. 

*Based on these investigations, 64 gives smoothest data without losing real features*

## Figure to illustrate smoothing process

In [12]:
trials1.index(92)

22

In [29]:
i = 22

head1 = rough[i]
smoothp = np.empty(np.shape(head1))
fr = 120
t = [each/fr for each in np.arange(np.shape(head1)[0])]
p = 1

for d in [0,1,2]:
    x = head1[:,d]
    GCV_auto = GCVSmoothedNSpline(t, x) #p chosen by program; check it is a good value.
    pf = GCV_auto.p * p
    GCV_manual = SmoothedNSpline(t, x, p=pf) #set to a multiple of the auto-selected value
    x_smoothed = GCV_manual(t)
    smoothp[:,d] = x_smoothed
    
#,25,41,44,47,50,53,62,63,64,65,
#,400,500,600,700,800
#,125,150,175

p_opts = [75,100,200,300]
smooths = []

for each in p_opts:    
    #create smoothed trajectories
    smoothp1 = np.empty(np.shape(head1))
    fr = 120
    t = [each/fr for each in np.arange(np.shape(head1)[0])]

    for d in [0,1,2]:
        x = head1[:,d]
        GCV_auto = GCVSmoothedNSpline(t, x) #p chosen by program
        pf = GCV_auto.p * each
        GCV_manual = SmoothedNSpline(t, x, p=pf) #set to a multiple of the auto-selected value
        x_smoothed = GCV_manual(t)
        smoothp1[:,d] = x_smoothed
        

    smooths.append(smoothp1)

plt.figure()
plt.suptitle(tn_list[i])
pos = smoothp


plt.scatter(head1[:,0],head1[:,2],s=10, label="Raw data")
plt.scatter(pos[:,0],pos[:,2], s=10, label="GCV spline")
plt.scatter(head1[32,0],head1[32,2],s=20, label="Head position 1")
plt.scatter(head1[51,0],head1[51,2],s=20, label="Head position 2")
for j in np.arange(len(p_opts)):
    plt.scatter(smooths[j][:,0],smooths[j][:,2],s=10,label=str(p_opts[j])+"x GCV")
        

plt.gca().set_ylim(min(pos[:,2])-0.10,max(pos[:,2])+0.10) #give a little breathing room around the data
plt.gca().set_aspect('equal')

plt.legend(loc='lower center',ncol=5)


## Apply Smoothing

In [12]:
#re-running all trials with selected smoothing parameter
smooth2 = []

for i in np.arange(len(rough)):
    head1 = rough[i]
    ind = tn_list.index(trials1[i])
    smoothp = np.empty(np.shape(head1))
    fr = 120
    t = [each/fr for each in np.arange(np.shape(head1)[0])]
    p = 65

    for d in [0,1,2]:
        x = head1[:,d]
        GCV_auto = GCVSmoothedNSpline(t, x) 
        pf = GCV_auto.p * p
        GCV_manual = SmoothedNSpline(t, x, p=pf) #set to a multiple of the auto-selected value
        x_smoothed = GCV_manual(t)
        smoothp[:,d] = x_smoothed
        
    smooth2.append(smoothp)
    np.savetxt(str(trials1[i])+'_'+ref['ID'][ind]+'_'+'smoothed.csv', smoothp, delimiter=",")

In [13]:
#graphing smoothed data on top of raw data

for t in np.arange(len(tn_list)):
    plt.figure()
    
    pos = smooth2[t]
    pos1 = rough[t]
    
    tf = transitions[t]
    af = accelerations[t]

    plt.suptitle(tn_list[t])
    plt.scatter(pos1[:,0],pos1[:,2],s=10,label = 'Unsmoothed',c="#A9A9A9",zorder=1)
    plt.plot(pos[:,0],pos[:,2], label='Smoothed (p = 64)',zorder=2)
    
    if not np.isnan(tf):
        tf = int(tf)
        plt.scatter(pos[tf,0],pos[tf,2],s=2,label='transition frame',zorder=3,c='#ff0000')

    plt.scatter(pos[af,0],pos[af,2],s=2,label='acceleration frame',zorder=6,c='#ff8282')

    plt.gca().set_ylim(min(pos[:,2])-0.10,max(pos[:,2])+0.10) #give a little breathing room around the data
    plt.gca().set_aspect('equal')

    plt.legend(loc='lower center',ncol=3)
    plt.savefig("/Users/grahmich/Desktop/Kinematics Paper/Graphs/X-Z Trajectories - Smoothed/"+str(tn_list[t])+"_headtraj_smoothed.pdf")
    plt.close()

# Create reference dataframe

## Position data

In [14]:
for ind in np.arange(len(smooth2)):
    oe1 = origins[ind][0] #origin position
    te1 = targets[ind][0] #target position
    
    
    #save origin and target branch locations
    ref.loc[ind,'orig_x'] = oe1[0]
    ref.loc[ind,'targ_x'] = te1[0]
    ref.loc[ind,'orig_y'] = oe1[1]
    ref.loc[ind,'targ_y'] = te1[1]
    ref.loc[ind,'orig_z'] = oe1[2]
    ref.loc[ind,'targ_z'] = te1[2]


In [15]:
ref.to_csv('/Users/grahmich/Desktop/Kinematics Paper/Dendrelaphis NC Data.csv',index=False)

# Velocity Metrics

In [16]:
#generate smoothed velocities.

for i in np.arange(len(smooth2)):
    
    ind = tn_list.index(trials1[i])
    smoothp = smooth2[i]
    trial = tn_list[ind]
    tf = accelerations[ind]

    #velocity metrics
    A,B = np.shape(smoothp)
    velocity = np.empty((A-2,B))
    
    for m in [0,1,2]:
        pos = smoothp[:,m]
        vel = np.empty(A-2)
        ts = np.asarray([k*1.0/fr for k in np.arange(len(vel))])
    
        for n in range(1,len(ts)+1):
            v = (pos[n+1]-pos[n-1])*fr/2.0
            vel[n-1] = v
    
        t = [each/fr for each in np.arange(np.shape(vel)[0])]
        GCV_auto = GCVSmoothedNSpline(t, vel)
        v_smoothed = GCV_auto(t)
        
        velocity[:,m] = v_smoothed

#   save values
    np.savetxt("/Users/grahmich/Desktop/Kinematics Paper/Processed files/"+str(trial)+ '_'+ref['ID'][ind]+'_'+'velocity.csv', velocity, delimiter=",")

### Velocity Check Code

In [17]:
check_trials = [t for i,j,t in zip(max_spot,max_spot1,tn_list) if not i-j==0 ]

#12 trials to check
plt.style.use('default')
pfiles = glob.glob('/Users/grahmich/Desktop/Kinematics Paper/Processed Files/*')
pfiles.sort()
velocity = [pfiles[i] for i in np.arange(len(pfiles)) if pfiles[i][-12:-4]=='velocity']
trials1 = [int(item[-25:-22]) for item in velocity]

n = 2
m = int(len(check_trials)/n)


fig, ax = plt.subplots(m,n,sharey=True)

for i in np.arange(m):
    for j in np.arange(n):
        t = i*n+j
        trial = check_trials[t]
        ind = trials1.index(trial)

        tf = transitions[ind]
        pos = np.linalg.norm(np.array(pd.read_csv(velocity[ind],header=None)),axis=1)
        ax[i,j].text(0.8, 0.8, velocity[ind][-25:-22], transform=ax[i,j].transAxes, ha="right")
        
        ax[i,j].plot(pos)
        ax[i,j].scatter(np.argmax(pos),max(pos),label='Max Overall')
        ax[i,j].scatter(np.argmax(pos[tf:])+tf,max(pos[tf:]),label='Max Lunge')
        ax[i,j].scatter(tf,pos[tf],label='Transition')
        ax[i,j].axhline(np.max(pos[tf:]))

            
handles, labels = ax[0,0].get_legend_handles_labels()
fig.legend(handles, labels,ncol=3,loc='upper center')

NameError: name 'max_spot' is not defined

## Trajectory Plots

### Below can be used for position or velocity in 3 dimensions.

In [32]:
#All head trajectories together
pfiles = glob.glob('/Users/Michelle/Desktop/Kinematics Paper/Trajectories/*')
pfiles.sort()
position = [pfiles[i] for i in np.arange(len(pfiles)) if pfiles[i][-12:-4]=='smoothed']
velocity = [pfiles[i] for i in np.arange(len(pfiles)) if pfiles[i][-12:-4]=='velocity']

dim = ['X','Y','Z']

#118 trials
m = 10
n = 12

fig, ax = plt.subplots(m,n,sharey=True)

for i in np.arange(m):
    for j in np.arange(n):
        t = i*n+j
        
        if t < len(position):
            pos = np.array(pd.read_csv(velocity[t],header=None))
            ax[i,j].text(0.8, 0.8, position[t][-25:-22], transform=ax[i,j].transAxes, ha="right")

            for d in [0,1,2]:
                ax[i,j].plot(pos[:,d],label=dim[d])
            
handles, labels = ax[0,0].get_legend_handles_labels()
fig.legend(handles, labels,ncol=3,loc='upper center')

### Below can be used for ploting a X-Z position view of a particular head trajectory

In [40]:
# Single graphs: X-Z position view
fig, ax = plt.subplots()

t=a.index('350a')
trial = position[t][-26:-22]
ind = tn_list.index(trial)
start = ref.loc[ind]['Start Frame']
end = ref.loc[ind]['End Frame']
trans = ref.loc[ind]['Transition frame']
acc = ref.loc[ind]['Acceleration frame']

orig = np.array(ref.loc[ind,['orig_x','orig_y','orig_z']])
targ = np.array(ref.loc[ind,['targ_x','targ_y','targ_z']])

snake = pd.read_csv(position[t],header=None)
frames = np.arange(start,end)



ax.scatter(snake[0],snake[2],s=4,c='#2ca02c',label='Head Trajectory')
ax.scatter(snake[0,trans],snake[2,trans],s=6,label = 'Transition')
ax.scatter(snake[0,acc],snake[2,acc],s=6, label = 'Acceleration')
ax.scatter(orig[0],orig[2],c='#1f77b4', label='Origin')
ax.scatter(targ[0],targ[2],c='#ff7f0e', label='Target')

ax.set_aspect('equal', 'box')
ax.set_title('Trial '+str(trial))
ax.set_xlabel('X position (m)')
ax.set_ylabel('Z position (m)')

ax.legend()

# fig.savefig(trial+'_head_traj.png', transparent=True)

Text(0, 0.5, 'Z position (m)')

#### Make a gif!

In [93]:
from matplotlib import animation

x = snake[0]
z = snake[2]

particle = np.vstack((x, z))

def update_lines(num, dataLines, lines) :
    for line, data in zip(lines, dataLines) :
        line.set_data(data[0:2, num-1:num])
    return lines


fig1, ax1 = plt.subplots()
ax1.set_title(trial+" - head trajectory")
ax1.scatter(orig[0],orig[2])
ax1.scatter(targ[0],targ[2])

#make animation data
data = [particle]
lines = [ax1.plot(data[0][0,0:1], data[0][1,0:1], 'o', color = 'k')[0]]
m = len(x)


# Creating the Animation object
ani = animation.FuncAnimation(fig1, update_lines, m, fargs=(data, lines),
                              interval=33, blit=False, repeat_delay = 1000)

ax1.set_aspect('equal', 'box')
ax1.set_ylim(min(z)-0.1,max(z)+0.1)
plt.show()

In [94]:
f = r"/Users/grahmich/Desktop/118.mp4" 
writervideo = animation.FFMpegWriter(fps=60) 
ani.save(f, writer=writervideo)

# Comparison Graphs

In [3]:
#Chrysopelea head trajectories

#Step 1: create trial list of trials where total < 98.5
cp_total_comp = cp_df[cp_df['total']<98.5]
comp_trials = cp_total_comp['tn'].values
all_trials = [each['tn'] for each in cp_ref]
comp_gs = np.around(np.array(cp_total_comp['gsr'].values),1)
comp_total = np.around(np.array(cp_total_comp['total'].values),1)

#Step 2: graph head trajectory for each of the comp trials

for each in comp_trials:
    ind = all_trials.index(each)
    t = list(comp_trials).index(each)
    gsr = comp_gs[t]
    total = comp_total[t]
    
    snake = sm_pos[ind]/1000.0
    
    # Single graphs: X-Z position view
    fig, ax = plt.subplots()
    ax.scatter(snake[:,0,0],snake[:,0,2],s=4,c='#2ca02c',label='Head Trajectory')
    
    ax.set_aspect('equal', 'box')
    ax.set_title('Trial '+str(each)+': GS = '+str(gsr)+'% SVL, DT = '+str(total)+' %SVL')
    ax.set_xlabel('X position (m)')
    ax.set_ylabel('Z position (m)')
    
    fig.savefig(str(each)+'_cp_head_traj.png', transparent=True)
    plt.close()

NameError: name 'cp_df' is not defined

# Looking at all trajectories in GS order

In [184]:
gaps_DP = []
for ind in np.arange(len(smooth2)):
    trial = tn_list[ind]
    smoothp = smooth2[ind]
    tf = transitions[ind]
    
    #gap size and SVL
    oe1 = origins[ind]
    te1 = targets[ind]
    svl = ref.loc[ind]['SVL']/100 #cm to m
    gsr = np.linalg.norm(te1-oe1)/svl*100
    gaps_DP.append(gsr)
    
gaps_CP = [all_imported[i]['gs_%'] for i in comp_inds]

A = sorted(zip(gaps_DP,tn_list))
B = sorted(zip(gaps_CP,comp_inds))

new_tns_DP = [thing[1] for thing in A]
new_inds_CP = [thing[1] for thing in B]

In [33]:
m = 10
n = 12

fig, ax = plt.subplots(m,n)

for i in np.arange(m):
    for j in np.arange(n):
        t = i*n+j
        if t < len(position):
            trial = new_tns_DP[t]
            ind = tn_list.index(trial)

            tf = transitions[ind]        
            pos = smooth1[ind]

            lp,hp = CP_Points(pos, tf)
            lp1,hp1 = DP_Points(pos, tf)

            ax[i,j].text(0.8, 0.8, tn_list[ind], transform=ax[i,j].transAxes, ha="right")

            ax[i,j].scatter(pos[:,0],pos[:,2],s=3)

            ax[i,j].axes.xaxis.set_visible(False)
            ax[i,j].axes.yaxis.set_visible(False)
            ax[i,j].set_aspect('equal')
            ax[i,j].set_ylim(min(pos[:,2])-0.1, max(pos[:,2])+0.1)

fig.suptitle('Dendrelaphis - Head Trajectories - Increasing Gap Size')

m = 10
n = 9

fig, ax = plt.subplots(m,n)

for i in np.arange(m):
    for j in np.arange(n):
        t = i*n+j
        ind = new_inds_CP[t]
        
        pos = sm_pos[ind][:,0,:] #head only
        frames = list(all_imported[ind]['fn'])
        dp = all_imported[ind]['dp']
        dpi = frames.index(int(dp))
        lp, hp = CP_Points(pos,dpi)
        lp1, hp1 = DP_Points(pos,dpi)

        
        ax[i,j].text(0.8, 0.8, all_imported[ind]['tn'], transform=ax[i,j].transAxes, ha="right")

        ax[i,j].scatter(pos[:,0],pos[:,2],s=3)
     

        ax[i,j].axes.xaxis.set_visible(False)
        ax[i,j].axes.yaxis.set_visible(False)
        ax[i,j].set_aspect('equal')
        ax[i,j].set_ylim(min(pos[:,2])-100, max(pos[:,2])+100)

fig.suptitle('Chrysopelea - Head Trajectories - Increasing Gap Size')


NameError: name 'new_tns_DP' is not defined

## Misc. Checks with processed trials

### Gap marker position 

In [109]:
# origin should be close to head position at frame 0 except for a few trials. 

#110 trials
m = 10
n = 11

fig, ax = plt.subplots(m,n)

for i in np.arange(m):
    for j in np.arange(n):
        t = i*n+j
        trial = int(tns[t])
        ind = tn_list.index(trial)

        origin = np.array([ref['orig_x'][ind],ref['orig_y'][ind],ref['orig_z'][ind]])
        target = np.array([ref['targ_x'][ind],ref['targ_y'][ind],ref['targ_z'][ind]])
        
        ax[i,j].scatter(origin[0],origin[2],label='origin')
        ax[i,j].scatter(target[0],target[2],label='target')
        ax[i,j].scatter(pos[t][:,0],pos[t][:,2],s=2,label='head')
        ax[i,j].scatter(pos[t][0,0],pos[t][0,2],s=3,label='start')
        
        ax[i,j].text(0.8, 0.8, trial, transform=ax[i,j].transAxes, ha="right")
        
        ax[i,j].set_ylim(-0.5,0.5)
        ax[i,j].set_xlim(origin[0]-0.1,target[0]+0.1)
        
        
        ax[i,j].axes.xaxis.set_visible(False)
        ax[i,j].axes.yaxis.set_visible(False)

        
handles, labels = ax[0,0].get_legend_handles_labels()
fig.legend(handles, labels,ncol=4,loc='upper center')

### Gap Distance Error Calculations

In [39]:
#gap size stuff
df = pd.read_csv('new_reference88.csv')

In [111]:
df.head()

,Trial,Snake,SVL,Mass,Start frame,Transition frame,Acceleration frame,End frame,Reference frame,att_gs,...,orig_z,targ_z,gsm,gsr,lands,maxs,avgs,relmax,relland,relavg
0,52,2018DP03,88.97,84.8,1932,3770,3788,3803,1900,43.0,...,0.228186,0.240296,0.424761,47.742017,0.408433,0.408433,0.113740,0.456350,0.456350,0.127084
1,56,2018DP03,88.97,84.8,7460,7470,7586,7606,4500,47.0,...,0.229285,0.237304,0.463046,52.045218,1.529387,1.529387,0.285720,1.708812,1.708812,0.319241
2,57,2018DP03,88.97,84.8,10988,11590,11615,11630,10980,45.0,...,0.230633,0.230700,0.443930,49.896627,0.710615,0.711053,0.127405,0.794473,0.793983,0.142352
3,59,2018DP03,88.97,84.8,2532,2930,2947,2957,2500,41.0,...,0.232787,0.216667,0.404141,45.424417,0.479467,0.479467,0.171952,0.535717,0.535717,0.192126
4,60,2018DP03,88.97,84.8,2659,3557,3570,3607,2500,42.0,...,0.230973,0.227726,0.417813,46.961084,1.269342,1.442358,0.143308,1.611573,1.418259,0.160121


In [115]:
diff_x = []
diff_z = []
diff_gs= []

gs = []

for i in np.arange(len(df)):
    targ = df.iloc[i][['targ_x','targ_y','targ_z']]
    orig = df.iloc[i][['orig_x','orig_y','orig_z']]
    
    diffs = [abs(targ[t]-orig[t]) for t in [0,1,2]]
    
    if np.isnan(df.iloc[i]['att_gs']):
        d = np.nan
    
    else:
        d = abs(df.iloc[i]['gsm']-df.iloc[i]['att_gs']/100.0)
        
    diff_x.append(abs(diffs[0]-df.iloc[i]['att_gs']/100.0))
    diff_z.append(diffs[2])
    diff_gs.append(d)
    gs.append(df.iloc[i]['gsm'])

In [116]:
plt.plot(diff_x,'o', label="X difference")
plt.plot(diff_z,'o', label = "Z difference")
plt.plot(diff_gs,'o', label = "Total difference")

place = np.arange(len(gs))
n = [df.iloc[i]['Trial'] for i in place]

plt.legend()
plt.title("Difference between actual and intended gap sizes (m)")

for i, txt in enumerate(n):

    plt.annotate(txt, (place[i], diff_gs[i]))

In [48]:
plt.axhline(0.04)
plt.axhline(-0.04)

In [118]:
#calculate percent errors
p_x = [abs(diff_x[i]/gs[i]*100) for i in np.arange(len(gs))]
p_z = [abs(diff_z[i]/gs[i]*100) for i in np.arange(len(gs))]
p_g = [abs(diff_gs[i]/gs[i]*100) for i in np.arange(len(gs))]

plt.figure()
plt.plot(p_x,'o', label="X difference")
plt.plot(p_z,'o', label = "Z difference")
plt.plot(p_g,'o', label = "Total difference")

plt.legend()
plt.title("% error between actual and intended gap sizes (m)")

for i, txt in enumerate(n):

    plt.annotate(txt, (place[i], p_x[i]))

In [119]:
plt.axhline(10)

In [120]:
L = [np.nanmean(thing) for thing in [p_x,p_z,p_g]]
L1 = [np.nanmean(thing) for thing in [diff_x,diff_z,diff_gs]]
print(L)
print(L1)

[3.345705359704625, 2.550528288527782, 3.327544596880871]
[0.01281484588901367, 0.010594706714889298, 0.012745535751270534]


### Testing different smoothing parameters

In [29]:
#what auto-pvals are being generated?

all_pvals = []

for i in np.arange(len(Files)):
    
    ID = Files[i][-19:-11] #read in as a string.
    trial = Files[i][-24:-20] #read in as a string.
    data = pd.read_csv(Files[i])
    
    ind = tn_list.index(trial)
    svl = ref.loc[ind]['SVL(cm)']/100 #cm to m
    
    start = ref.loc[ind]['Start Frame']
    end = ref.loc[ind]['End Frame']
    frame = ref.loc[ind]['Reference frame']
    
   
    #pull out relevant points: gravity, origin and target ends. 
    ghigh = data.columns[-3:]    
    glow = data.columns[-6:-3]
    
    g1 = data[glow].loc[frame-1].values
    g2 = data[ghigh].loc[frame-1].values
    grav = g1-g2
    gnorm = grav/np.linalg.norm(grav)
    
    OE = np.array(data.loc[[frame-1],['pt2_X','pt2_Y','pt2_Z']])
    TE = np.array(data.loc[[frame-1],['pt3_X','pt3_Y','pt3_Z']])
    
    
    #nan check #1
    if np.isnan(np.sum(g2)):
        print(str(trial)+": ghigh has nans")
        continue
        
    #nan check # 2
    if np.isnan(np.sum(g1)):
        print(str(trial)+": glow has nans")
        continue
    
    #nan check 3
    if np.isnan(np.sum(OE)):
        print(str(trial)+": OE has nans")
        continue
        
    #nan check 4
    if np.isnan(np.sum(TE)):
        print(str(trial)+": glow has nans")
        continue 
        
    OE = OE.reshape(1,3)
    TE = TE.reshape(1,3)
    g1 = g1.reshape(1,3)
    g2 = g2.reshape(1,3)

    
    #crop head data to area of interest, between gap entry and landing.
    head_traj = np.array(data[['pt1_X','pt1_Y','pt1_Z']])
    head_traj = head_traj[start:end,:]
    
    #nan check 5
    if np.isnan(np.sum(head_traj)):
        print(str(trial)+": head_traj has nans")
        continue
    
    #align coordinates
    oe1 = fix_coords(OE,OE,TE,gnorm)
    te1 = fix_coords(TE,OE,TE,gnorm)
    g1a = fix_coords(g1,OE,TE,gnorm)
    g2a = fix_coords(g2,OE,TE,gnorm)

    
    head1 = fix_coords(head_traj,OE,TE,gnorm)
    
    #create smoothed trajectories
    smoothp = np.empty(np.shape(head1))
    fr = 120
    t = [each/fr for each in np.arange(np.shape(head1)[0])]

    pfs = []
    for d in [0,1,2]:
        x = head1[:,d]
        GCV = GCVSmoothedNSpline(t, x) #p chosen by program; check it is a good value.
        x_smoothed = GCV(t)
        smoothp[:,d] = x_smoothed
        pfs.append(GCV.p)
        
    all_pvals.append(pfs)
    
flatt = [item for sublist in all_pvals for item in sublist]
max(flatt),min(flatt),np.mean(flatt)